In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def fetch_historical_xrp_data(from_date, to_date):
    # Define parameters
    symbol = "XRPUSDT"
    interval = "1m"
    
    # Convert from_date and to_date to timestamp in milliseconds (UTC)
    start_time = int(datetime.strptime(from_date, "%Y-%m-%d").replace(hour=0, minute=0, second=0, microsecond=0).timestamp() * 1000)
    end_time = int(datetime.strptime(to_date, "%Y-%m-%d").replace(hour=23, minute=59, second=59, microsecond=999).timestamp() * 1000)

    # Initialize a list to hold all data
    all_data = []

    # Fetch historical data in chunks
    while start_time < end_time:
        url = f"https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&startTime={start_time}&endTime={end_time}&limit=1000"
        response = requests.get(url)
        data = response.json()

        if not data or isinstance(data, dict) and 'msg' in data:  # Check for error messages
            print("Error fetching historical data:", data)
            break

        all_data.extend(data)

        # Update start_time to the last timestamp fetched
        start_time = int(data[-1][6]) + 1  # Move to the next minute after the last data point's Close Time

    # Prepare DataFrame
    df = pd.DataFrame(all_data, columns=["Open Time", "Open", "High", "Low", "Close", "Volume", 
                                          "Close Time", "Quote Asset Volume", "Number of Trades", 
                                          "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore"])

    # Convert Open Time to readable format (UTC)
    df["Open Time"] = pd.to_datetime(df["Open Time"], unit='ms')

    # Convert UTC to Gulf Time (UTC+4)
    df["Open Time"] = df["Open Time"] + timedelta(hours=4)

    # Select relevant columns
    df = df[["Open Time", "Close"]].rename(columns={"Open Time": "Timestamp", "Close": "XRP Price"})

    # Filter data to only include rows between the specified dates (optional, if additional filtering is needed)
    df_filtered = df[(df["Timestamp"].dt.date >= pd.to_datetime(from_date).date()) &
                     (df["Timestamp"].dt.date <= pd.to_datetime(to_date).date())]

    # Print the filtered data
    print(df_filtered)

    # Save to CSV (optional)
    file_path = f"xrp_data_{from_date}_to_{to_date}_gulf_time.csv"
    df_filtered.to_csv(file_path, index=False)
    print(f"Data saved to {file_path}")

# Example usage
fetch_historical_xrp_data(from_date="2024-10-18", to_date="2024-10-21")


Error fetching historical data: []
               Timestamp   XRP Price
0    2024-10-18 00:00:00  0.54430000
1    2024-10-18 00:01:00  0.54430000
2    2024-10-18 00:02:00  0.54500000
3    2024-10-18 00:03:00  0.54570000
4    2024-10-18 00:04:00  0.54580000
...                  ...         ...
3905 2024-10-20 17:05:00  0.53910000
3906 2024-10-20 17:06:00  0.53910000
3907 2024-10-20 17:07:00  0.53910000
3908 2024-10-20 17:08:00  0.53910000
3909 2024-10-20 17:09:00  0.53910000

[3910 rows x 2 columns]
Data saved to xrp_data_2024-10-18_to_2024-10-21_gulf_time.csv
